In [4]:
import requests
from bs4 import BeautifulSoup, Comment

In [5]:
url = 'http://sibgrapi.sid.inpe.br/col/sid.inpe.br/banon/2001/03.30.15.38.24/doc/mirror.cgi/About?languagebutton=en&languagerep1=sid.inpe.br/banon/2001/05.04.15.05&languagerep2=dpi.inpe.br/banon/1999/05.03.22.11&firstlanguagerep=sid.inpe.br/banon/2001/05.04.15.05&submissionformrep=dpi.inpe.br/banon-pc2@80/2006/08.30.19.42&returnbutton=yes'
req_main_page = requests.get(url)

In [6]:
main_page = BeautifulSoup(req_main_page.content, 'html.parser')
table = main_page.find_all('table')

In [7]:
def getYearConf(string):
    counter = 0
    year = ""
    for char in string:
        if char.isdigit():
            year += char
            counter += 1
        if(counter == 4):
            return year

In [8]:
years = []
all_links = []
for tag_a in table[0].find_all('a'):
    year = getYearConf(tag_a.text)
    link = tag_a.get('href')
    dois_pontos = ":"
    format = dois_pontos + str(year) + dois_pontos
    if year not in years and year != None:
        years.append(year)
        all_links.append(link)
    elif year in years and year != None:
        if year == '2021':
            length = len(all_links)
            all_links[length - 1] = link

In [9]:
confs = []
for i in range(len(years)):
    lista = [int(years[i]), all_links[i]]
    confs.append(lista)

In [10]:
main_pages_papers_confs = []
for conf in confs:
    req_main_page_paper = requests.get(conf[1])
    main_page_paper = BeautifulSoup(req_main_page_paper.content, 'html.parser')
    frame = main_page_paper.find_all('frame')
    yearLinks = [conf[0], frame[1].get('src')]
    main_pages_papers_confs.append(yearLinks)

In [91]:
def getTagsAfterComments(link):
    req_main_page_paper = requests.get(link)
    main_page_paper = BeautifulSoup(req_main_page_paper.content, 'html.parser')
    separator = "aaaaaaaaaaaa"
    comment = main_page_paper.find(string=lambda text: isinstance(text, Comment) and separator in text)
    if comment:
        next_element = comment.find_next_siblings()
        if next_element:
            return next_element
        else:
            return None
                    

In [99]:
def getQtPapersCategory(html):
    countersCategory = {}
    key = ""
    counterPapers = 0
    for tag in html:
        if tag.get('class') and "titleAuthorTABLE" in tag.get('class'):
            counterPapers = counterPapers + 1
        if tag.name == "b":
            if counterPapers == 0:
                key = tag.text
            countersCategory[key] = counterPapers
            counterPapers = 0
            key = tag.text
    
    countersCategory[key] = counterPapers
    return countersCategory

In [141]:
def getLinksPapers(qt, html):
    links_papers = {}
    counter = 0
    all_links = []
    key_html = ""
    for tag in html:
        if tag.get('class') and "titleAuthorTABLE" in tag.get('class'):
            all_links.append(tag.find('a').get('href'))
            counter = counter + 1
        if tag.name == "b":
            if counter == 0:
                key_html = tag.text
            if qt[key_html] == counter:
                links_papers[key_html] = all_links
                all_links = []
                counter = 0
            key_html = tag.text
        
    links_papers[key_html] = all_links
    for key, links in links_papers.items():
        for url in links:
            url_formatted = "http://" + url
            req_paper_page = requests.get(url_formatted)
            print(req_paper_page)
            paper_page = BeautifulSoup(req_paper_page.content, 'html.parser')
            frames = paper_page.find_all("frame")
            print(frames)
        


In [143]:
# Fazer um for
html_links_paper = getTagsAfterComments(main_pages_papers_confs[0][1])

if html_links_paper:
    qt_papers_category = getQtPapersCategory(html_links_paper)
    getLinksPapers(qt_papers_category, html_links_paper)

else:
    print("No links to paper")

ConnectionError: HTTPConnectionPool(host='goto', port=80): Max retries exceeded with url: /sid.inpe.br/sibgrapi/2022/09.12.07.42?ibiurl.backgroundlanguage=pt-BR&searchinputvalue=&parentidentifiercitedby=8JMKD3MGPEW34M/495MHJ8+6qtX3pFwXQZeBBx/pBvav&forcerecentflag=0&forcehistorybackflag=0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fcca9d1ec20>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))